### Parameter

In [1]:
import open3d as o3d
import numpy as np
import os
import copy
import ICP_utilities as uti

#parameters
model_path = r'C:\Users\dings\Documents\Masterarbeit\models'
stl_path= os.path.join(model_path,'obergesenk.stl')
ply_path= os.path.join(model_path,'obergesenk_gescannt.ply') 
source = o3d.io.read_point_cloud(ply_path)
mesh = o3d.io.read_triangle_mesh(stl_path)
target = o3d.geometry.PointCloud(mesh.vertices)
trans_init=[[1,0,0,0],
            [0,1,0,0],    
            [0,0,1,0],            
            [0,0,0,1],]

num_iteration = 50
threshold = 30
threshold_calib = 0.01



Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Calibration

In [2]:
if __name__ == "__main__":
    #low_noise_source = uti.rauchen_eliminieren(source,nb_points=25,radius=3)
    down_sample_source = source.uniform_down_sample(every_k_points=3)
    #----step by step----:
    #registed_source,target = uti.icp_algo_step_by_step(source=source, target=target, threshold=threshold, trans_init=trans_init, num_iteration=num_iteration)
    #----quick show----:
    registed_source,target = uti.icp_algo(source=source, target=target, threshold=threshold, trans_init=trans_init,max_iteration=num_iteration)
    
    #----calibration----:
    registed_source,target = uti.calibration_after_rough_reg(source=registed_source,target=target,threshold=threshold_calib,num_samples=5)
    o3d.io.write_point_cloud(os.path.join(model_path,"reg_source.ply"),registed_source)
    o3d.io.write_point_cloud(os.path.join(model_path,"reg_target.ply"),target)
    o3d.io.write_point_cloud(os.path.join(model_path,"registed_both.ply"),registed_source+target)
    #uti.compute_dists(source=registed_source,target=target,tolerance=0.4)
    print("3 point clouds saved in path: %s"%(model_path))
    
    
    

finetune registration after the rough registration
Visualization of two rough registed point clouds before finetune
press 'q' to continue

chose No. 1 coordinate of 5 total in source and press 'q' to continue

1) Please pick points using [shift + left click]
   Press [shift + right click] to undo point picking
2) After picking points, press 'Q' to close the window
[Open3D INFO] Picked point #69553 (27., 19., -1.1e+02) to add in queue.
chose No. 1 coordinate of 5 total in target and press 'q' to continue

1) Please pick points using [shift + left click]
   Press [shift + right click] to undo point picking
2) After picking points, press 'Q' to close the window
[Open3D INFO] Picked point #125072 (27., 20., -1.1e+02) to add in queue.
chose No. 2 coordinate of 5 total in source and press 'q' to continue

1) Please pick points using [shift + left click]
   Press [shift + right click] to undo point picking
2) After picking points, press 'Q' to close the window
[Open3D INFO] Picked point #5889

### Segmentation

In [3]:
    print("segmentieren...")
    o3d.visualization.draw_geometries_with_editing([registed_source])
    print("cropped file saved in path:%s"%(model_path))


segmentieren...
cropped file saved in path:C:\Users\dings\Documents\Masterarbeit\models


### Comparison

In [ ]:
    #Set cropped reference json coodinates
    reference_json_cood = os.path.join(model_path,"cropped_3.json")
    uti.segments_compare(registed_source,target,reference_json_cood,tolerance=0.4)